In [1]:
import json
import pandas as pd

In [2]:
def save2json(file_path, data):
    """
    Salvo un dic a un json

    Parameters
    ----------
    file_path: str
      Path al json
    data: dic
      Diccionario a guardar
    """
    with open(file_path, 'w') as data_file:
        json.dump(data, data_file, indent=4)

In [3]:
xl = pd.ExcelFile('Escuelas_20161207092310413.xlsx')
xl.sheet_names

['Escuelas']

In [4]:
df = xl.parse("Escuelas")

In [5]:
df.columns

Index(['codigo_empresa', 'numero_anexo', 'id_tipo_empresa', 'tipo_empresa',
       'establecimiento', 'direccion_nivel', 'cue', 'cue_anexo',
       'codigo_inspeccion', 'id_empresa_padre', 'id_escuela_madre',
       'id_dependencia', 'dependencia', 'id_tipo_escuela', 'tipo_escuela',
       'privado', 'id_ambito_escuela', 'ambito_escuela',
       'id_categoria_escuela', 'categoria_escuela', 'id_tipo_jornada',
       'tipo_jornada', 'id_modalidad_jornada', 'modalidad_jornada',
       'hospitalaria', 'religioso', 'arancelado', 'albergue', 'colectivos',
       'latitud', 'longitud', 'tipo_educacion', 'nivel_educativo', 'domicilio',
       'barrio', 'id_localidad', 'localidad', 'id_departamento',
       'departamento', 'nro_registro', 'fec_ultima_modificacion'],
      dtype='object')

In [6]:
df['departamento'].unique()

array(['SANTA MARIA', nan, 'CAPITAL', 'CALAMUCHITA', 'COLON',
       'GENERAL ROCA', 'GENERAL SAN MARTIN', 'MARCOS JUAREZ',
       'PTE. ROQUE SAENZ PEÑA', 'PUNILLA', 'RIO CUARTO', 'RIO SECO',
       'RIO SEGUNDO', 'SAN JUSTO', 'TERCERO ARRIBA', 'UNION',
       'JUAREZ CELMAN', 'SOBREMONTE', 'TULUMBA', 'TOTORAL', 'ISCHILIN',
       'CRUZ DEL EJE', 'SAN ALBERTO', 'SAN JAVIER', 'RIO PRIMERO', 'MINAS',
       'POCHO'], dtype=object)

In [7]:
df.latitud[0], type(df.latitud[0])
df = df.fillna(False)

In [8]:
df.shape

(6531, 41)

In [9]:
df = df[df.latitud != False]
df.shape

(5086, 41)

In [10]:
df.nivel_educativo.unique()

array(['SECUNDARIO', 'PRIMARIO', False, 'SUPERIOR', 'INICIAL', 'ESPECIAL'], dtype=object)

In [11]:
secundarios = df[df.nivel_educativo == 'SECUNDARIO']
secundarios.shape

(1057, 41)

In [12]:
features = []

departamentos_busqueda = ['CAPITAL', 'COLON', 'RIO PRIMERO', 'SANTA MARIA', 'RIO SEGUNDO', False]

for i in range(secundarios.shape[0]):
    if secundarios.iloc[i].departamento in departamentos_busqueda:
        
        long = secundarios.iloc[i]['longitud'].replace(',', '.')
        lat = secundarios.iloc[i]['latitud'].replace(',', '.')
        
        features.append({"geometry": {
        "coordinates": [float(long), float(lat)], 
        "type": "Point"
    },
 "properties": {
        'name': secundarios.iloc[i]['establecimiento'],
        'type': 'secondary',
        'tipo_escuela': secundarios.iloc[i]['tipo_empresa'],
        'ambito': secundarios.iloc[i]['ambito_escuela'],
        'nivel_educativo': secundarios.iloc[i]['nivel_educativo']
    },
 "type": "Feature"
})

len(features)

491

In [13]:
data = {"features": features,
        "type": "FeatureCollection"}

save2json('./../secundarios.geojson', data)

In [14]:
primarios = df[df.nivel_educativo == 'PRIMARIO']
primarios.shape

(2177, 41)

In [15]:
features = []

base = {"geometry": {
        "coordinates": [], 
        "type": "Point"
    },
 "properties": {
        'nombre': '',
        'tipo_escuela': '',
        'ambito': '',
        'nivel_educativo': ''
    },
 "type": "Feature"
}


departamentos_busqueda = ['CAPITAL', 'COLON', 'RIO PRIMERO', 'SANTA MARIA', 'RIO SEGUNDO', False]

for i in range(primarios.shape[0]):
    if primarios.iloc[i].departamento in departamentos_busqueda:
        
        long = primarios.iloc[i]['longitud'].replace(',', '.')
        lat = primarios.iloc[i]['latitud'].replace(',', '.')
        
        features.append({"geometry": {
        "coordinates": [float(long), float(lat)], 
        "type": "Point"
    },
 "properties": {
        'name': primarios.iloc[i]['establecimiento'],
        'type': 'primary',
        'tipo_escuela': primarios.iloc[i]['tipo_empresa'],
        'ambito': primarios.iloc[i]['ambito_escuela'],
        'nivel_educativo': primarios.iloc[i]['nivel_educativo']
    },
 "type": "Feature"})

len(features)

708

In [16]:
data = {"features": features,
        "type": "FeatureCollection"}

save2json('./../primarios.geojson', data)